In [1]:
import requests
import pandas as pd
from constants import Y_mapping, years_filtered, headers
from utils import process_calendar_year, process_crop_year  # Import functions

# List to store individual DataFrames
dfs_Ys = []

for product in Y_mapping:
    sector = product["sector"]
    year_type = product['year']
    HS_codes = product["HS codes"]
    product_name = product["name"]  # Access the name field

    #Hiding the source. Even though it's public - not many stakeholders aware of it. 
    url = f"XXXXXX&sectors={sector}&{HS_codes}"
    headers = headers

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json() 
    else:
        print("Failed to retrieve data")
        data = None

    if data:
        df = pd.DataFrame(data)
        
        if year_type == 'calendar':
            monthly_data = process_calendar_year(df,product_name)
        else:
            monthly_data = process_crop_year(df,product_name)
        
        monthly_data = monthly_data.rename(columns={'kgEquivalent': product_name})  # Rename column with product name
        dfs_Ys.append(monthly_data)
    else:
        print("No data to convert to DataFrame")

# Merge all DataFrames on 'date'
if dfs_Ys:
    merged_df = dfs_Ys[0]
    for df in dfs_Ys[1:]:
        merged_df = pd.merge(merged_df, df, on='date', how='outer').dropna()
    
    # Optionally rename the date column if needed
    merged_df = merged_df.rename(columns={'date': 'Date'})

    # Print the merged DataFrame
    print(merged_df.head(10))
else:
    print("No dataframes to merge")


        Date    TAXUD_SMP
0 2014-11-01  57017.12426
1 2014-12-01  63557.94567
2 2015-01-01  59881.75056
3 2015-02-01  56514.28904
4 2015-03-01  72528.12546
5 2015-04-01  58598.02309
6 2015-05-01  54430.25917
7 2015-06-01  62359.12788
8 2015-07-01  50562.91617
9 2015-08-01  64058.01096


In [2]:
# main_script.py
import os
from database_connection import query_data


a_ssh_host = "XXXX"
a_ssh_user = "XXXX"
a_ssh_port = "XXXX"
a_ssh_private_key = "XXXX"
a_sql_hostname = "XXXX"
a_sql_username = "XXXX"
a_sql_password = "XXXX"
a_sql_database = "XXXX"
a_sql_port = "XXXX"

In [3]:
from constants import X_mapping
import pandas as pd
import os
from dotenv import load_dotenv
from database_connection import ssh_tunnel, db_connection, query_data

# Load environment variables from .env
load_dotenv()


dfs_Xs = []

with ssh_tunnel(a_ssh_host, a_ssh_port, a_ssh_user, a_ssh_private_key, a_sql_hostname, a_sql_port) as local_port:
    with db_connection(local_port, a_sql_username, a_sql_password, a_sql_database) as conn:
        for item in X_mapping:
            X_code = item['X_code']
            name = item['name']

            query = f'''SELECT date, amount
                        FROM XXXX
                        WHERE country_id = 200 AND hs_code = {X_code} AND data_interval = 'monthly' AND type = 2 AND date > '2010-12-31'
                        ORDER BY date ASC'''

            result = query_data(conn, query)
            result['date'] = pd.to_datetime(result['date'])
            result = result.rename(columns={'amount': f'X_{name}'})

            dfs_Xs.append(result)

if dfs_Xs:
    merged_df = dfs_Xs[0]
    for df in dfs_Xs[1:]:
        merged_df = pd.merge(merged_df, df, on='date', how='outer').dropna()
    
    merged_df = merged_df.rename(columns={'date': 'Date'})
    print(merged_df.head(10))
else:
    print("No dataframes to merge")

        Date   GTT_SMP
0 2011-01-01  40959.67
1 2011-02-01  36941.64
2 2011-03-01  49733.05
3 2011-04-01  44068.32
4 2011-05-01  45951.41
5 2011-06-01  41445.08
6 2011-07-01  40523.06
7 2011-08-01  42305.13
8 2011-09-01  50473.71
9 2011-10-01  57250.27


In [4]:
all_dfs = dfs_Ys + dfs_Xs

def merge_dfs_on_date(df_list):
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = pd.merge(merged_df, df, on='date', how='outer')
    return merged_df

# Merge all dataframes
merged_df = merge_dfs_on_date(all_dfs).dropna()

merged_df = pd.DataFrame(merged_df)

print(merged_df.head(5))

        date    TAXUD_SMP   GTT_SMP
0 2014-11-01  57017.12426  60794.67
1 2014-12-01  63557.94567  62611.84
2 2015-01-01  59881.75056  66746.20
3 2015-02-01  56514.28904  58408.43
4 2015-03-01  72528.12546  67607.55


In [5]:
import matplotlib.pyplot as plt


merged_df['year'] = merged_df['date'].dt.year

# Group by year and sum the values
annual_data = merged_df.groupby('year').sum().reset_index()

# Flatten the MultiIndex columns
annual_data.columns = ['year'] + [f'annual_{col}' for col in annual_data.columns if col != 'year']

pd.set_option('display.float_format', '{:.2f}'.format) 


def percentage_difference(col1, col2):
    return (col1 - col2) / col2 * 100

suffixes = set(col.split('_')[1] for col in merged_df.columns if '_' in col)

yearly_stats_list = []



for suffix in suffixes:
    taxud_col = f'annual_Y_{suffix}'
    gtt_col = f'annual_X_{suffix}'
    if taxud_col in annual_data.columns and gtt_col in annual_data.columns:
        diff_col = f'%_diff_{suffix}'
        annual_data[diff_col] = percentage_difference(annual_data[taxud_col], annual_data[gtt_col])

        yearly_stats_list.append(annual_data[['year',diff_col]])

if yearly_stats_list:
    annual = yearly_stats_list[0]
    for df in yearly_stats_list[1:]:
        annual = pd.merge(annual, df, on='year')
    
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(annual)
else:
    print("No dataframes to merge")




    year  %_diff_SMP
0   2014       -2.29
1   2015       -6.39
2   2016       -5.01
3   2017       -5.01
4   2018        2.88
5   2019       -4.37
6   2020       -6.95
7   2021       -4.69
8   2022       -7.23
9   2023       -7.15
10  2024       -2.74


In [6]:
import pandas as pd

def percentage_difference(col1, col2):
    return (col1 - col2) / col2 * 100


suffixes = set(col.split('_')[1] for col in merged_df.columns if '_' in col)

merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month

filtered_df_1 = merged_df[merged_df['year'].isin(years_filtered)]
monthly_stats_list = []

for suffix in suffixes:
    taxud_col = f'Y_{suffix}'
    gtt_col = f'X_{suffix}'
    if taxud_col in merged_df.columns and gtt_col in merged_df.columns:
        diff_col = f'%_diff_{suffix}'
        merged_df[diff_col] = percentage_difference(merged_df[taxud_col], merged_df[gtt_col])
        filtered_df_1[diff_col] = merged_df[diff_col]  # Ensure the column is also in filtered_df_1

        if diff_col in filtered_df_1.columns:
            
            monthly_stats = filtered_df_1.groupby('month')[diff_col].agg(['mean', 'min', 'max']).reset_index()

            monthly_stats.columns = ['month', f'average_percentage_difference_{suffix}', f'min_percentage_difference_{suffix}', f'max_percentage_difference_{suffix}']

            monthly_stats[f'maximum_error_{suffix}'] = monthly_stats.apply(
                lambda row: max(abs(row[f'min_percentage_difference_{suffix}']), abs(row[f'max_percentage_difference_{suffix}'])) - abs(row[f'average_percentage_difference_{suffix}']),
                axis=1
            )

            monthly_stats_list.append(monthly_stats[['month', f'maximum_error_{suffix}']])

if monthly_stats_list:
    monthly = monthly_stats_list[0]
    for df in monthly_stats_list[1:]:
        monthly = pd.merge(monthly, df, on='month', how='outer')
    
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(monthly)
else:
    print("No dataframes to merge")


    month  maximum_error_SMP
0       1              18.40
1       2              14.50
2       3              10.51
3       4              24.20
4       5              10.12
5       6              14.09
6       7              12.77
7       8              15.10
8       9               8.89
9      10              16.59
10     11               7.52
11     12              13.66


Probability of worst case scenario